In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-product-reviews/Reviews.csv
/kaggle/input/flipkart-product-review-dataset/flipkart_product.csv
/kaggle/input/flipkart-product-customer-reviews-dataset/Dataset-SA.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

2025-11-02 05:49:19.567815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762062559.750159      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762062559.802220      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Flipkart Product Reviews
flipkart_df1 = pd.read_csv("/kaggle/input/flipkart-product-review-dataset/flipkart_product.csv", encoding='latin1')
flipkart_df2 = pd.read_csv("/kaggle/input/flipkart-product-customer-reviews-dataset/Dataset-SA.csv")

# Amazon Product Reviews
amazon_df = pd.read_csv("/kaggle/input/amazon-product-reviews/Reviews.csv")

In [7]:
print(flipkart_df1.head())
print(flipkart_df2.head())
print(amazon_df.head())

                                         ProductName     Price Rate  \
0  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
1  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    5   
2  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   
3  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    1   
4  Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...  ??3,999    3   

            Review                                            Summary  
0           Super!  Great cooler.. excellent air flow and for this...  
1          Awesome             Best budget 2 fit cooler. Nice cooling  
2             Fair  The quality is good but the power of air is de...  
3  Useless product                 Very bad product it's a only a fan  
4             Fair                                      Ok ok product  
                                        product_name product_price Rate  \
0  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999

In [11]:
# Combine Flipkart datasets 
flipkart_df = pd.concat([flipkart_df1, flipkart_df2], ignore_index=True)

# Flipkart
flipkart_df.rename(columns={'Review': 'reviewText', 'Rate': 'rating'}, inplace=True)

# Amazon
amazon_df.rename(columns={'Text': 'reviewText', 'Score': 'rating'}, inplace=True)

# Combine Flipkart + Amazon
combined_df = pd.concat([flipkart_df, amazon_df], ignore_index=True)

In [12]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)           # remove punctuation
    text = text.strip()
    return text

combined_df['review_clean'] = combined_df['reviewText'].astype(str).apply(clean_text)


In [15]:
# Convert to numeric, coerce errors to NaN
combined_df['rating'] = pd.to_numeric(combined_df['rating'], errors='coerce')

# Optional: drop rows where rating could not be converted
combined_df = combined_df.dropna(subset=['rating'])

# Ensure integers
combined_df['rating'] = combined_df['rating'].astype(int)


In [16]:
# rating <= 2 → risky (1), else neutral (0)
combined_df['label'] = combined_df['rating'].apply(lambda x: 1 if x <= 2 else 0)

# Check distribution
print(combined_df['label'].value_counts())


label
0    827703
1    135669
Name: count, dtype: int64


In [17]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(combined_df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=combined_df['label'])
print(dict(zip(classes, class_weights)))


{0: 0.5819551215834665, 1: 3.5504499922605754}


In [18]:
combined_df[combined_df['label']==1][['reviewText','rating']].head(5)


,reviewText,rating
3,Useless product,1
8,Unsatisfactory,1
32,Expected a better product,2
48,Waste of money!,1
59,Worthless,1


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    combined_df['review_clean'].tolist(),
    combined_df['label'].tolist(),
    test_size=0.1,
    random_state=42,
    stratify=combined_df['label']
)